## imports

In [2]:
import pandas as pd
import os
import json

In [3]:
import re
import copy

In [ ]:
#find "#old style" for other type of texts separators

## READ THE FILE

In [128]:
path = 'lighttag_annotations_2001-2099_rows.json'
filename, file_extension = os.path.splitext(path)
print(filename)
print(file_extension)

lighttag_annotations_2001-2099_rows
.json


In [74]:
raw_data = []
for line in open(path, 'r', encoding='utf-8'):
    raw_data.append(json.loads(line))
    annotations_and_examples = []
    relations = []
    #у нас джейсон в одну линию записан, поэтому по сути annannotations_and_examples и relations не обнуляются, и накапливают всю
    #инфу, которую надо
    for d in raw_data:
        for k, v in d.items():
            if k == 'annotations_and_examples':
                annotations_and_examples.append(v)
            if k == 'relations':
                relations.append(v)
                
#we have no relations in our task, so we need not them to append, so we need not the_list


In [75]:
#annotations_and_examples contains:

# content
# metadata
# annotations 
# classifications


In [76]:
raw_data = []
for line in annotations_and_examples[0]:
    raw_data.append((line))
    content = []
    metadata = []
    annotations = []
    classifications = []
    for d in raw_data:
        for k, v in d.items():
            if k == 'content':
                content.append(v)
            if k == 'metadata':
                metadata.append(v)
            if k == 'annotations':
                annotations.append(v)
            if k == 'classifications':
                classifications.append(v)

In [77]:
#size = 1
size = len(content)
print(size)

2


In [78]:
annotations[0][186]

{'start': 43187, 'end': 43194, 'tag': 'м'}

In [79]:
annotations[size-1][0]
#annotations[3][0]

{'start': 93, 'end': 108, 'tag': 'п'}

In [80]:
content[size-1]

"['\\n############\\n2301,Бывшего главу администрации Переваловского муниципального образования Эдуарда Егунова приговорили к двум годам лишения свободы за аферы с землей . Его взяли под стражу в зале суда . Как сообщает пресс-служба прокуратуры Тюменской области , 51-летний экс-чиновник был признан виновным сразу по трем статьям УК: Превышение должностных полномочий , Мошенничество и Покушение на причинение имущественного ущерба собственнику путем обмана в особо крупном размере . По версии следствия , Егунов в период с 2005 по 2014 годы будучи главой поселка , а затем главой его администрации , под видом дачной амнистии незаконно вывел из государственной собственности 26 земельных участков общей площадью почти 70 тыс . квадратных метров и распределил их по своему усмотрению . Ущерб , нанесенный Тюменскому муниципальному району , куда входит поселок Перевалово , и государству превысил 4 млн рублей . Как полагает следствие , глава поселка изготавливал фиктивные постановления администрац

## READ THE DATA

## TAGS

In [81]:
starts = []
ends = []
tags = []

for i in range(len(annotations)):
    raw_data = []
    for line in annotations[i]:
        raw_data.append((line))
        st = []
        en = []
        tg = []
        for d in raw_data:
            for k, v in d.items():
                if k == 'start':
                    st.append(v)
                if k == 'end':
                    en.append(v)
                if k == 'tag':
                    tg.append(v)
    starts.append(st)
    ends.append(en)
    tags.append(tg)

len(starts)

2

In [82]:
len(starts[size-1])

497

In [83]:
content[size-1][starts[size-1][15]:ends[size-1][15]]

'Москву '

###  пустышка, заполняем нулями

In [84]:
tagged_content = copy.copy(content)

In [85]:
print(len(tagged_content) == len(content), '\n')

for i in range(0, size):
    print(len(tagged_content[i])==len(content[i]))

True 

True
True


In [86]:
for j in range(0, size):
    list_content = list(tagged_content[j])

    for i in range(0, len(list_content)):
        #old style
        #if list_content[i] != ' ' and list_content[i] != '#' and list_content[i] != '\n':
        if list_content[i] != ' ' and list_content[i] != '#' and list_content[i] != 'n' and list_content[i] != '\\':
            list_content[i] = 'O'

    tagged_content[j] = "".join(list_content)

In [87]:
print(len(tagged_content) == len(content), '\n')

for i in range(0, size):
    print(len(tagged_content[i])==len(content[i]))

True 

True
True


## Избегаем схлопывания многословных тегов: все, что в промежутке тега, заполняем его индикатором, но если этот промежуток совпал с пробелом в оригинальном контенте - тут тоже пробел

In [88]:
for q in range(0, size):
    list_content = list(tagged_content[q])

    for i in range(0, len(tags[q])):
        for j in range(starts[q][i], ends[q][i]):
            if content[q][j] == ' ':
                list_content[j] = ' '
            else:
                list_content[j] = tags[q][i]

    tagged_content[q] = "".join(list_content)

In [89]:
print(len(tagged_content) == len(content), '\n')

for i in range(0, size):
    print(len(tagged_content[i])==len(content[i]))

True 

True
True


## хлопаем получившиеся посхедовательности до одного символа

In [90]:
for i in range(0, len(tagged_content)):
    tagged_content[i] = re.sub(r'O+', 'O', tagged_content[i])
    tagged_content[i] = re.sub(r'ж+', 'ж', tagged_content[i])
    tagged_content[i] = re.sub(r'п+', 'п', tagged_content[i])
    tagged_content[i] = re.sub(r'д+', 'д', tagged_content[i])
    tagged_content[i] = re.sub(r'м+', 'м', tagged_content[i])

In [91]:
for i in range(0, size):
    bull = (len(tagged_content[i].split(' ')) == len(content[i].split(' ')))
    print(bull, '\n')

True 

True 



## TAGS AND TEXT BOTH

## Разделяем на тексты и теги к текстам

In [92]:
list_of_tags = []
sub_list_of_tags = []
for i in range(0, len(tagged_content)):
    sub_list_of_tags.append(tagged_content[i].split('\\n############\\n'))
    #old style
    #sub_list_of_tags.append(tagged_content[i].split('\n############\n'))

for i in range(0, size):
    list_of_tags+=sub_list_of_tags[i]

In [93]:
list_of_texts = []
sub_list_of_texts = []
for i in range(0, len(content)):
    sub_list_of_texts.append(content[i].split('\\n############\\n'))
    #old style
    #sub_list_of_texts.append(content[i].split('\n############\n'))

    
for i in range(0, size):
    list_of_texts+=sub_list_of_texts[i]

In [94]:
print(len(list_of_texts))
print(len(list_of_tags))
#список со всеми текстами и список со всеми текстами тегов, элемент - один текст строкой или один текст тегов строкой

101
101


In [95]:
list_of_text_tokens = []
for i in range(0, len(list_of_texts)):
    list_of_text_tokens.append([list_of_texts[i].split(' ')])

In [96]:
list_of_tags_tokens = []
for i in range(0, len(list_of_tags)):
    list_of_tags_tokens.append([list_of_tags[i].split(' ')])

In [97]:
# в целом те же самые списки, только теперь элемент - список токенов или список тегов (строки разбили на отдельные элементы)

## Проверка совпадения количество тегов - токенов

In [98]:
for i in range(0, len(list_of_text_tokens)):
    if len(list_of_tags_tokens[i][0])!=len(list_of_text_tokens[i][0]):
        print('idx ', i)
        print(len(list_of_tags_tokens[i][0]))
        print(len(list_of_text_tokens[i][0]))
        print('\n')

## Проверка соответствия тегов токенам

In [99]:
#проверяем по-токеново idx-й текст
idx = 30
for i in range(0, len((list_of_text_tokens[idx][0]))):
    if list_of_tags_tokens[idx][0][i] !='O':
        print('слово ',list_of_text_tokens[idx][0][i])
        print('тег ',list_of_tags_tokens[idx][0][i])
        print('\n')

слово  Испании
тег  м


слово  Михаиле
тег  п


слово  К
тег  п


слово  .
тег  п


слово  Ольге
тег  п


слово  К
тег  п


слово  с
тег  д


слово  2007
тег  д


слово  по
тег  д


слово  2009
тег  д


слово  годы
тег  д


слово  Ольга
тег  п


слово  К
тег  п


слово  .
тег  п


слово  Испании
тег  м


слово  Ольгу
тег  п


слово  К
тег  п


слово  Михаила
тег  п


слово  К
тег  п


слово  Сергей
тег  п


слово  М
тег  п


слово  Олег
тег  п


слово  Н
тег  п




## Переворачиваем в нормальные теги

In [100]:
final_list_of_tags_tokens = []
for i in range(0, len(list_of_tags_tokens)):
    sublist = []
    for j in range(0, len(list_of_tags_tokens[i][0])):
        if list_of_tags_tokens[i][0][j] == 'п':
            sublist.append('OFF')
        elif list_of_tags_tokens[i][0][j] == 'ж':
            sublist.append('SUF')
        elif list_of_tags_tokens[i][0][j] == 'д':
            sublist.append('DAT')
        elif list_of_tags_tokens[i][0][j] == 'м':
            sublist.append('LOC')
        else:
            sublist.append('O')
    final_list_of_tags_tokens.append(sublist)
    

In [101]:
len(final_list_of_tags_tokens)

101

In [102]:
len(final_list_of_tags_tokens[0])

1

In [103]:
len(list_of_tags_tokens[0][0])

1

## Заносим все в один датафрейм, подтягиваем столб с оригинальной датой, сохр csv

In [104]:
final = []
for i in range(0, len(list_of_text_tokens)):
    final.append([list_of_text_tokens[i][0], final_list_of_tags_tokens[i]])

In [105]:
str_final = []
for i in range(1, len(final)):
    str_text = ' '.join(final[i][0])
    str_tags = ' '.join(final[i][1])
    str_final.append([str_tags, str_text])
str_final[0][0]

'O O OFF OFF OFF OFF O OFF OFF O O O O O O O O O O O O O O O O O O O O O O O O OFF O O DAT DAT DAT DAT DAT DAT O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O LOC LOC O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O OFF OFF OFF O O O O O O'

### удаляем номер текста и его айдишник (и внешние кавычки) - ужасные артефакты

In [106]:
len(str_final[0][1].split(" "))

142

In [107]:
len(str_final[0][0].split(" "))

142

In [108]:
#old style
#len(''.join(str_final[0][1].split(",")[1:-1]).split(' '))
len(','.join(str_final[0][1].split(",")[1:-2]).split(' '))

141

In [109]:
str_final[30][1].split(' ')

['2261,Сумма',
 'ущерба',
 'от',
 'действий',
 'руководящих',
 'сотрудников',
 'Министерства',
 'культуры',
 'России',
 'может',
 'превышать',
 '1',
 'млн',
 'руб',
 '.',
 'Об',
 'этом',
 'Интерфаксу',
 'сообщил',
 'информированный',
 'источник',
 '.',
 'Речь',
 'идет',
 'о',
 'миллионах',
 ',',
 'точная',
 'сумма',
 'будет',
 'установлена',
 'позднее',
 ',',
 '—',
 'сказал',
 'собеседник',
 'агентства',
 '.',
 'Он',
 'пояснил',
 ',',
 'что',
 'уголовное',
 'дело',
 'о',
 'хищениях',
 'расследуется',
 'по',
 'статье',
 'мошенничество',
 'в',
 'особо',
 'крупном',
 'размере',
 '(',
 'ч',
 '.',
 '4',
 'ст',
 '.',
 '159',
 'УК',
 'РФ',
 ')',
 '.',
 'По',
 'его',
 'словам',
 ',',
 'следователи',
 'работают',
 'более',
 'чем',
 'по',
 'десяти',
 'адресам',
 '.',
 'Утром',
 '15',
 'марта',
 'стало',
 'известно',
 ',',
 'что',
 'ФСБ',
 'возбудила',
 'уголовное',
 'дело',
 'против',
 'руководящих',
 'работников',
 'Минкультуры',
 'и',
 'ряда',
 'предпринимателей',
 '.',
 'Их',
 'подозревают',


In [110]:
','.join(str_final[30][1].split(",")[1:-2]).split(' ')

['Сумма',
 'ущерба',
 'от',
 'действий',
 'руководящих',
 'сотрудников',
 'Министерства',
 'культуры',
 'России',
 'может',
 'превышать',
 '1',
 'млн',
 'руб',
 '.',
 'Об',
 'этом',
 'Интерфаксу',
 'сообщил',
 'информированный',
 'источник',
 '.',
 'Речь',
 'идет',
 'о',
 'миллионах',
 ',',
 'точная',
 'сумма',
 'будет',
 'установлена',
 'позднее',
 ',',
 '—',
 'сказал',
 'собеседник',
 'агентства',
 '.',
 'Он',
 'пояснил',
 ',',
 'что',
 'уголовное',
 'дело',
 'о',
 'хищениях',
 'расследуется',
 'по',
 'статье',
 'мошенничество',
 'в',
 'особо',
 'крупном',
 'размере',
 '(',
 'ч',
 '.',
 '4',
 'ст',
 '.',
 '159',
 'УК',
 'РФ',
 ')',
 '.',
 'По',
 'его',
 'словам',
 ',',
 'следователи',
 'работают',
 'более',
 'чем',
 'по',
 'десяти',
 'адресам',
 '.',
 'Утром',
 '15',
 'марта',
 'стало',
 'известно',
 ',',
 'что',
 'ФСБ',
 'возбудила',
 'уголовное',
 'дело',
 'против',
 'руководящих',
 'работников',
 'Минкультуры',
 'и',
 'ряда',
 'предпринимателей',
 '.',
 'Их',
 'подозревают',
 'в',

In [111]:
for i in range(0, len(str_final)):
    str_final[i][1] = ','.join(str_final[i][1].split(",")[1:-2])
    #эта строка появилась потому что при моем стиле формирования текстов на разметку в конце каждого разделителя выползает кавычка, которой присваивается собственный тег
    #вот он-то сейчас и удалится
    str_final[i][0] = ' '.join(str_final[i][0].split(' ')[:-1])

In [112]:
#old style
#for i in range(0, len(str_final)):
#    str_final[i][1] = ','.join(str_final[i][1].split(",")[1:-1])

In [113]:
#old style
#for i in range(0, len(str_final)):
#    str_final[i][1] = str_final[i][1].replace('"', '')

In [114]:
len(str_final[0][0].split(' '))

141

In [115]:
len(str_final[0][1].split(' '))

141

## все, датафрейм

In [116]:
df = pd.DataFrame(str_final, columns=['tags', 'texts'])
df.head()

,tags,texts
0,O O OFF OFF OFF OFF O OFF OFF O O O O O O O O ...,Бывшего директора ООО Тюменская продовольствен...
1,O LOC O O O O O O O O O O O O O O O O O O O O ...,"В Тюмени вынесли приговор аферистам , пытавшим..."
2,O O O O O O O O OFF OFF O O O O O O O O O O O ...,Как сообщил РБК-Татарстан адвокат Равиль Тукта...
3,O O O LOC O O O O O O O O O O O O OFF OFF O O ...,Ряд сотрудников мэрии Самары подозреваются в м...
4,O O O O OFF OFF OFF OFF O O O O O O O O O O O ...,Глава американской фармацевтической компании T...


In [117]:
df['texts'][0]

'Бывшего директора ООО Тюменская продовольственная база 44-летнего Александра Бронникова признали виновным в мошенничестве , совершенном в особо крупном размере . Суд приговорил мужчину к 5 годам лишения свободы . По версии следствия , Бронников в период с апреля по ноябрь 2012 года брал у бизнесменов и своих знакомых в долг крупные суммы под видом привлечения инвестиций для развития бизнеса и закупа рыба . Однако деньги он так и не вернул . По делу признано 11 потерпевших ( физических и юридических лиц ) , размер причиненного им ущерба составил более 65 , 5 млн рублей , сообщает пресс-служба прокуратуры Тюменской области . Пострадавшие в 2013 году обратились в правоохранительные органы , было возбуждено уголовное дело . Суд удовлетворил 7 исков потерпевших , взыскав с осужденного в их пользу более 52 млн рублей . ,Экс-директора Тюменской продовольственной базы осудили за хищение 65 млн'

In [118]:
for i in range(0, len(df)):
    if len(df['texts'][i].split(' '))!=len(df['tags'][i].split(' ')):
        print('idx ', i)
        print(df['texts'][i])
        print(len(df['texts'][i].split(' ')))
        print('\n')
        print(df['tags'][i])
        print(len(df['tags'][i].split(' ')))
        print('\n')

idx  48
Ранее судимый 32-летний житель Самары после освобождения из мест лишения приобрел несколько сотовых телефонов и сим-карт . После этого , узнав коды регионов Татарстана , мошенник начал обзванивать людей и вымогать у них деньги . Преступник говорил , что их родственник попал в беду и за решение вопроса о не возбуждении уголовного дела необходима определенная сумма денег . Потерпевшие деньги перечисляли на номера его сотовых телефонов , а он тратил их на собственные нужды . В ходе проведения оперативно-розыскных мероприятий сотрудниками полиции подозреваемый был задержан и арестован . В общей сложности ущерб составил около 150000 рублей . 
98


O O O O LOC O O O O O O O O O O O O O O O O O O LOC O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O LOC O O O
102


idx  99
Калининский районный суд Новосибирска приговорил экс-начальника юридического отдела ПО Север Владислава Зуева к 6 , 5

In [119]:
df.to_csv(filename+'_parsed_data_str.csv', index = False)

### проверяем формат строк и все такое при подгрузке сформированного файла обратно

In [120]:
df1 = pd.read_csv(filename+'_parsed_data_str.csv')
df1.head()

,tags,texts
0,O O OFF OFF OFF OFF O OFF OFF O O O O O O O O ...,Бывшего директора ООО Тюменская продовольствен...
1,O LOC O O O O O O O O O O O O O O O O O O O O ...,"В Тюмени вынесли приговор аферистам , пытавшим..."
2,O O O O O O O O OFF OFF O O O O O O O O O O O ...,Как сообщил РБК-Татарстан адвокат Равиль Тукта...
3,O O O LOC O O O O O O O O O O O O OFF OFF O O ...,Ряд сотрудников мэрии Самары подозреваются в м...
4,O O O O OFF OFF OFF OFF O O O O O O O O O O O ...,Глава американской фармацевтической компании T...


In [125]:
list_of_tags = []
for i in range(0, len(df1['tags'])):
    if i!=49:
        list_of_tags.append(df1['tags'][i].split(' '))

list_of_texts = []
for i in range(0, len(df1['texts'])):
    if i!=49:
        list_of_texts.append(df1['texts'][i].split(' '))


In [126]:
for i in range(0, len(list_of_texts)):
    if len(list_of_tags[i])!=len(list_of_texts[i]):
        print('i: ',i)
        print('diff: ',len(list_of_texts[i])- len(list_of_tags[i]))

i:  48
diff:  -4
i:  98
diff:  -1


## MAKING FILES FOR DEEP PAVLOV TRAIN NER

In [61]:
file = open('train.txt','w', encoding='utf-8') 
for i in range(0, int(0.8*len(list_of_texts))):
    file.write('-DOCSTART-' + '\t'+'O'+'\n')
    for j in range(0, len(list_of_texts[i])):
        line = list_of_texts[i][j] + '\t' + list_of_tags[i][j] + '\n'
        file.write(line)
        if list_of_texts[i][j] == '.':
            file.write('\n')
            file.write('\n')
file.close()

In [62]:
file = open('test.txt','w', encoding='utf-8') 
for i in range(int(0.8*len(list_of_texts)), int(0.9*len(list_of_texts))):
    file.write('-DOCSTART-' + '\t'+'O'+'\n')
    for j in range(0, len(list_of_texts[i])):
        line = list_of_texts[i][j] + '\t' + list_of_tags[i][j] + '\n'
        file.write(line)
        if list_of_texts[i][j] == '.':
            file.write('\n')
            file.write('\n')
file.close()

In [63]:
file = open('valid.txt','w', encoding='utf-8') 
for i in range(int(0.9*len(list_of_texts)), len(list_of_texts)):
    file.write('-DOCSTART-' + '\t'+'O'+'\n')
    for j in range(0, len(list_of_texts[i])):
        line = list_of_texts[i][j] + '\t' + list_of_tags[i][j] + '\n'
        file.write(line)
        if list_of_texts[i][j] == '.':
            file.write('\n')
            file.write('\n')
file.close()